In [1]:
# from google.colab import drive

In [2]:
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

In [4]:
cd 'C:\Users\alans\OneDrive\Desktop\Sem5\CSE3105\J component\twitter-suicidal-intention-dataset'

[WinError 123] The filename, directory name, or volume label syntax is incorrect: "'C:\\Users\\alans\\OneDrive\\Desktop\\Sem5\\CSE3105\\J component\\twitter-suicidal-intention-dataset'"
c:\Users\alans\OneDrive\Desktop\Sem5\CSE3105\J component


In [5]:
#!git clone https://github.com/laxmimerit/twitter-suicidal-intention-dataset.git

#Pre-Processing

In [6]:
import re
import json
from bs4 import BeautifulSoup
import unicodedata

In [19]:
abbreviations_path= "\Abbreviations\abbreviations.json"

def abbr_to_full(x):
	abbreviations = json.load(open(abbreviations_path))

	if type(x) is str:
		for key in abbreviations:
			value = abbreviations[key]
			raw_text = r'\b' + key + r'\b'
			x = re.sub(raw_text, value, x)
		return x
	else:
		return x

In [20]:
def remove_emails(x):
	return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x)

In [21]:
def remove_urls(x):
	return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x)

In [22]:
def remove_html_tags(x):
	return BeautifulSoup(x, 'lxml').get_text().strip()

In [23]:
def remove_rt(x):
	return re.sub(r'\brt\b', '', x).strip()

In [24]:
def remove_special_chars(x):
	x = re.sub(r'[^\w ]+', "", x)
	x = ' '.join(x.split())
	return x

In [25]:
def remove_accented_chars(x):
	x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
	return x

In [26]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = abbr_to_full(x)
    x = remove_emails(x)
    x = remove_urls(x)
    x = remove_html_tags(x)
    x = remove_rt(x)
    x = remove_accented_chars(x)
    x = remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

#Twitter Dataset

In [29]:
df1= pd.read_csv('\twitter-suicidal-intention-dataset\twitter-suicidal_data.csv')

OSError: [Errno 22] Invalid argument: '\twitter-suicidal-intention-dataset\twitter-suicidal_data.csv'

In [ ]:
df1.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [ ]:
df1['intention'].value_counts()

0    5121
1    3998
Name: intention, dtype: int64

In [ ]:
df1['tweet'] = df1['tweet'].apply(lambda x:get_clean(x))

In [ ]:
# !mkdir /content/drive/MyDrive/ML Project/Cleaned_Data

In [ ]:
#df1.to_csv("/gdrive/MyDrive/SuicideTendencyDetection/Cleaned_Data/twitterData.csv")

In [ ]:
tfidf = TfidfVectorizer(max_features= 50000, ngram_range = (1,3),analyzer = 'char') 

In [ ]:
X = tfidf.fit_transform(df1['tweet'])
y = df1['intention']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state = 42)

In [ ]:
clf = SVC()

In [ ]:
clf.fit(X_train, y_train)

SVC()

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1019
           1       0.96      0.88      0.92       805

    accuracy                           0.93      1824
   macro avg       0.93      0.93      0.93      1824
weighted avg       0.93      0.93      0.93      1824



In [ ]:
x = 'death is an old friend'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([0])

In [ ]:
import pickle
with open('suicideDetecttion.pkl', 'wb') as fid:
  pickle.dump(clf, fid)

In [ ]:
with open('suicideDetecttion.pkl', 'rb') as fid:
  mod = pickle.load(fid)

In [ ]:
mod.predict(vec)

array([0])


# Reddit Dataset



In [ ]:
df2= pd.read_csv('/content/drive/MyDrive/ML Project/kaggle/Suicide_Detection.csv', encoding = 'unicode_escape')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df2.head()

,Unnamed: 0,text,class,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Am I weird I don't get affected by compliments...,non-suicide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,i need helpjust help me im crying so hard,suicide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,"Iâm so lostHello, my name is Adam (16) and I...",suicide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f = ['text', 'class']
df2 = df2[f]
df2.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide


In [ ]:
df2['class']=df2['class'].replace('suicide', 1)
df2['class']=df2['class'].replace('non-suicide', 0)

In [ ]:
df2=df2.dropna()

In [ ]:
df2 = df2[ (df2['class'] ==1) | (df2['class'] == 0)]

In [ ]:
df2["class"].unique()

array([1, 0], dtype=object)

In [ ]:
df2.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,"Iâm so lostHello, my name is Adam (16) and I...",1


In [ ]:
df2['text'] = df2['text'].apply(lambda x:get_clean(x))

In [ ]:
df2.to_csv('/content/drive/MyDrive/ML Project/Cleaned_Data/redditData.csv')

In [ ]:
tfidf = TfidfVectorizer(max_features= 100000, ngram_range = (1,5),analyzer = 'char') 

In [ ]:
X = tfidf.fit_transform(df2['text'])
y = df2['class']

In [ ]:
y=y.astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state = 42)

In [ ]:
clf = SVC()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
x = "My cat passed away and I miss him"
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

#Combining Twitter and Reddit Dataset

In [ ]:
df1= pd.read_csv("/gdrive/MyDrive/SuicideTendencyDetection/Cleaned_Data/twitterData.csv")

In [ ]:
df2= pd.read_csv("/gdrive/MyDrive/SuicideTendencyDetection/Cleaned_Data/redditData.csv")

In [ ]:
df1.rename(columns = {'tweet':'text', 'intention':'class'}, inplace = True)
df1=df1[["text", "class"]]
df2=df2[["text", "class"]]

In [ ]:
print(df1.shape)
print(df2.shape)

(9119, 2)
(232044, 2)


In [ ]:
df3 = pd.concat([df1, df2], ignore_index=True, sort=False)

In [ ]:
df3.head()

,text,class
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [ ]:
df3.shape

(241163, 2)

In [ ]:
df3.to_csv("/gdrive/MyDrive/SuicideTendencyDetection/Cleaned_Data/consolidatedData.csv")

#Consolidated- Training 



In [ ]:
#if colab
df3= pd.read_csv("/gdrive/MyDrive/SuicideTendencyDetection/Cleaned_Data/consolidatedData.csv")

In [ ]:
#if windows
#df3= pd.read_csv("C:/Users/Harisha/Downloads/consolidatedData.csv")

In [ ]:
df3.head()

,Unnamed: 0,Unnamed: 0.1,text,class
0,0,0,my life is meaningless i just want to end my l...,1
1,1,1,muttering i wanna die to myself daily for a fe...,1
2,2,2,work slave i really feel like my only purpose ...,1
3,3,3,i did something on the 2 of october i overdose...,1
4,4,4,i feel like no one cares i just want to die ma...,1


In [ ]:
df3.shape

(241163, 4)

In [ ]:
#tfidf = TfidfVectorizer(max_features= 100000, ngram_range = (1,5),analyzer = 'char') 

In [ ]:
#df3.dropna()

,Unnamed: 0,Unnamed: 0.1,text,class
0,0,0,my life is meaningless i just want to end my l...,1
1,1,1,muttering i wanna die to myself daily for a fe...,1
2,2,2,work slave i really feel like my only purpose ...,1
3,3,3,i did something on the 2 of october i overdose...,1
4,4,4,i feel like no one cares i just want to die ma...,1
...,...,...,...,...
241158,241158,241158,if you do not like rock then your not going to...,0
241159,241159,241159,you how you can tell i have so many friends an...,0
241160,241160,241160,pee probably tastes like salty teaa14i can som...,0
241161,241161,241161,the usual stuff you find hereim not posting th...,1


In [ ]:
#X = tfidf.fit_transform(df3['text'].apply(lambda x: np.str_(x)))
y = df3['class']

In [ ]:
#import pickle
#with open('vectorized_X.pkl', 'wb') as fid:
#  pickle.dump(X, fid)

In [ ]:
import pickle
with open('vectorized_X.pkl', 'rb') as fid:
  X = pickle.load(fid)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state = 42)

#Training LinearSVC

In [ ]:
clf = LinearSVC()

In [ ]:
clf.fit(X_train, y_train)

LinearSVC()

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95     24309
           1       0.95      0.95      0.95     23924

    accuracy                           0.95     48233
   macro avg       0.95      0.95      0.95     48233
weighted avg       0.95      0.95      0.95     48233



In [ ]:
import pickle
with open('suicideDetecttion_SVM.pkl', 'wb') as fid:
  pickle.dump(clf, fid)

#Training MLP

In [ ]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 3), random_state=1)

In [ ]:
mlp.fit(X_train, y_train)

C:\Users\Harisha\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 3), random_state=1,
              solver='lbfgs')

In [ ]:
y_pred = mlp.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95     24309
           1       0.94      0.95      0.94     23924

    accuracy                           0.94     48233
   macro avg       0.94      0.94      0.94     48233
weighted avg       0.94      0.94      0.94     48233



In [ ]:
import pickle
with open('suicideDetecttion_MLP.pkl', 'wb') as fid:
  pickle.dump(mlp, fid)

#Training MultinomialNB

In [ ]:
nb= MultinomialNB()

In [ ]:
nb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_pred = nb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     24309
           1       0.86      0.95      0.91     23924

    accuracy                           0.90     48233
   macro avg       0.91      0.90      0.90     48233
weighted avg       0.91      0.90      0.90     48233



In [ ]:
import pickle
with open('suicideDetecttion_NB.pkl', 'wb') as fid:
  pickle.dump(nb, fid)

#Training SGDClassifier

In [ ]:
sgd= SGDClassifier()

In [ ]:
sgd.fit(X_train, y_train)

SGDClassifier()

In [ ]:
y_pred= sgd.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24309
           1       0.93      0.93      0.93     23924

    accuracy                           0.93     48233
   macro avg       0.93      0.93      0.93     48233
weighted avg       0.93      0.93      0.93     48233



In [ ]:
import pickle
with open('suicideDetecttion_SGD.pkl', 'wb') as fid:
  pickle.dump(sgd, fid)#Training SGDClassifier

#Training AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier()

In [ ]:
abc.fit(X_train, y_train)

In [ ]:
y_pred= abc.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
abcs = AdaBoostClassifier(SVC(probability=True, kernel='linear'))

In [ ]:
abcs.fit(X_train, y_train)